### Nutri-score Project - Decision Modeling - Master BDMA 2025/2026

**Notebook 0: Data Collection & Processing**

This notebook collects food data from Open Food Facts database and creates an Excel spreadsheet containing the data. It also cleans the data and validates if it fits the minimum requirements for the project.

**Requirements:**
- At least 200 products
- Each Nutri-Score label (A-E): at least 15% representation
- Each Green-Score label (A-E): at least 10% representation
- No duplicates by product name

In [13]:
import pandas as pd
import numpy as np
import requests
import time
from typing import List, Dict
import model_utils
import _1_nutri_score as ns

#### Collect Alternatives via Open Food Facts API

In [14]:
def search_products_by_category(category: str, n_products: int = 250, 
                                country: str = 'france') -> List[Dict]:
    """
    Search products from Open Food Facts API by category
    
    Parameters:
    - category: food category (e.g., 'breakfast-cereals', 'cookies', 'pizzas')
    - n_products: number of products to fetch
    - country: country code for products
    
    Returns list of product dictionaries
    """
    base_url = "https://world.openfoodfacts.org/cgi/search.pl"
    
    products = []
    page = 1
    page_size = 50
    
    print(f"Fetching {n_products} products from category: {category}")
    
    while len(products) < n_products:
        params = {
            'action': 'process',
            'tagtype_0': 'categories',
            'tag_contains_0': 'contains',
            'tag_0': category,
            'tagtype_1': 'countries',
            'tag_contains_1': 'contains',
            'tag_1': country,
            'page_size': page_size,
            'page': page,
            'json': 1
        }
        
        try:
            response = requests.get(base_url, params=params, timeout=240)
            data = response.json()
            
            if 'products' not in data or len(data['products']) == 0:
                print(f"No more products found at page {page}")
                break
            
            for product in data['products']:
                # Extract required nutritional information
                product_data = extract_product_data(product)
                if product_data and is_valid_product(product_data):
                    products.append(product_data)
                    
                if len(products) >= n_products:
                    break
            
            page += 1
            time.sleep(0.5)  # Be respectful to API
            
            if len(products) % 50 == 0:
                print(f"Collected {len(products)} products...")
                
        except Exception as e:
            print(f"Error fetching page {page}: {e}")
            time.sleep(2)
            continue
    
    print(f"Successfully collected {len(products)} products")
    return products[:n_products]

def extract_product_data(product: Dict) -> Dict:
    """Extract relevant nutritional data from product JSON"""
    try:
        # Check if product has a name
        product_name = product.get('product_name', '').strip()
        if not product_name:
            return None  # Skip products without names
        
        nutriments = product.get('nutriments', {})
        
        # Get Nutri-Score
        nutri_grade = product.get('nutriscore_grade', '').upper()
        nutri_score = product.get('nutriscore_score', None)
        
        # Get Eco-Score (Green-Score)
        ecoscore_grade = product.get('ecoscore_grade', '').upper()
        ecoscore_score = product.get('ecoscore_score', None)
        
        data = {
            'product_name': product_name,
            'brands': product.get('brands', ''),
            'categories': product.get('categories', ''),
            'url': f"https://world.openfoodfacts.org/product/{product.get('code', '')}",
            
            # Nutritional values per 100g
            'energy_100g': nutriments.get('energy_prepared_100g', None) or nutriments.get('energy_100g', 0),
            'saturated_fat_100g': nutriments.get('saturated-fat_prepared_100g', None) or nutriments.get('saturated-fat_100g', 0),
            'sugars_100g': nutriments.get('sugars_prepared_100g', None) or nutriments.get('sugars_100g', 0),
            'salt_100g': nutriments.get('salt_prepared_100g', None) or nutriments.get('salt_100g', 0),
            'proteins_100g': nutriments.get('proteins_prepared_100g', None) or nutriments.get('proteins_100g', 0),
            'fiber_100g': nutriments.get('fiber_prepared_100g', None) or nutriments.get('fiber_100g', 0),
            'fvl_percent': nutriments.get('fruits-vegetables-legumes-estimate-from-ingredients_100g', 0),
            
            # Nutri-Score
            'nutri_score_value': nutri_score,
            'nutri_score_label': nutri_grade if nutri_grade in ['A','B','C','D','E'] else None,
            
            # Green-Score (Eco-Score)
            'green_score_value': ecoscore_score,
            'green_score_label': ecoscore_grade if ecoscore_grade in ['A','B','C','D','E'] else None,
        }
        
        return data
        
    except Exception as e:
        print(f"Error extracting product data: {e}")
        return None

def is_valid_product(product_data: Dict) -> bool:
    """Check if product has all required nutritional data"""
    required_fields = [
        'energy_100g', 'saturated_fat_100g', 'sugars_100g', 'salt_100g',
        'proteins_100g', 'fiber_100g', 'nutri_score_label', 'green_score_label'
    ]
    
    for field in required_fields:
        if product_data.get(field) is None:
            return False
        if field in ['nutri_score_label', 'green_score_label']:
            if product_data[field] not in ['A', 'B', 'C', 'D', 'E']:
                return False
    
    return True

#### Balancing Dataset

In [15]:
def balance_dataset(df: pd.DataFrame, 
                   min_nutri_pct: float = 0.15,
                   min_green_pct: float = 0.10) -> pd.DataFrame:
    """
    Balance dataset to meet project requirements by downsampling overrepresented labels
    """
    # Check current distribution
    print("\n=== Current Distribution ===")
    print("\nNutri-Score:")
    print(df['nutri_score_label'].value_counts().sort_index())
    print("\nGreen-Score:")
    print(df['green_score_label'].value_counts().sort_index())
    
    # Calculate target counts
    n_products = len(df)
    nutri_target = int(n_products * min_nutri_pct)
    green_target = int(n_products * min_green_pct)
    
    print(f"\n=== Target Counts ===")
    print(f"Minimum per Nutri-Score label: {nutri_target}")
    print(f"Minimum per Green-Score label: {green_target}")
    
    # Balance by undersampling or flagging needs
    nutri_counts = df['nutri_score_label'].value_counts()
    green_counts = df['green_score_label'].value_counts()
    
    # Identify which labels need more products
    print("\n=== Status Check (Before Balancing) ===")
    nutri_needs_more = []
    for label in ['A', 'B', 'C', 'D', 'E']:
        nutri_count = nutri_counts.get(label, 0)
        if nutri_count < nutri_target:
            nutri_status = f"⚠️  Need {nutri_target - nutri_count} more"
            nutri_needs_more.append(label)
        else:
            nutri_status = "✓"
        print(f"Nutri-Score {label}: {nutri_count} products - {nutri_status}")
    
    print()
    green_needs_more = []
    for label in ['A', 'B', 'C', 'D', 'E']:
        green_count = green_counts.get(label, 0)
        if green_count < green_target:
            green_status = f"⚠️  Need {green_target - green_count} more"
            green_needs_more.append(label)
        else:
            green_status = "✓"
        print(f"Green-Score {label}: {green_count} products - {green_status}")
    
    # AUTOMATIC BALANCING: Downsample overrepresented categories
    if nutri_needs_more or green_needs_more:
        print("\n=== Automatic Balancing ===")
        print("Downsampling overrepresented labels to balance the dataset...\n")
        
        # Calculate minimum count needed to satisfy both requirements
        min_needed_nutri = max(nutri_counts.get(label, 0) for label in nutri_needs_more) if nutri_needs_more else nutri_target
        min_needed_green = max(green_counts.get(label, 0) for label in green_needs_more) if green_needs_more else green_target
        
        # Find the minimum count across all labels
        all_nutri_counts = [nutri_counts.get(label, 0) for label in ['A', 'B', 'C', 'D', 'E']]
        all_green_counts = [green_counts.get(label, 0) for label in ['A', 'B', 'C', 'D', 'E']]
        
        min_nutri = min(all_nutri_counts)
        min_green = min(all_green_counts)
        
        # Strategy: Keep products that help balance both scores
        # Calculate new target based on minimum representation
        new_total_target = max(
            int(min_nutri / min_nutri_pct),  # Total needed to make min nutri = 15%
            int(min_green / min_green_pct),  # Total needed to make min green = 10%
            200  # Minimum project requirement
        )
        
        new_nutri_target = int(new_total_target * min_nutri_pct)
        new_green_target = int(new_total_target * min_green_pct)
        
        print(f"New target total products: {new_total_target}")
        print(f"New Nutri-Score target per label: {new_nutri_target}")
        print(f"New Green-Score target per label: {new_green_target}")
        
        # Downsample each combination to achieve balance
        balanced_dfs = []
        
        for nutri_label in ['A', 'B', 'C', 'D', 'E']:
            for green_label in ['A', 'B', 'C', 'D', 'E']:
                subset = df[(df['nutri_score_label'] == nutri_label) & 
                           (df['green_score_label'] == green_label)]
                
                if len(subset) > 0:
                    # Calculate how many from this combination we should keep
                    # This is a proportional allocation
                    balanced_dfs.append(subset)
        
        # Alternative simpler strategy: downsample overrepresented labels
        balanced_dfs = []
        for label in ['A', 'B', 'C', 'D', 'E']:
            nutri_subset = df[df['nutri_score_label'] == label]
            count = len(nutri_subset)
            
            # Keep at most 2x the target to maintain some variety
            max_keep = max(new_nutri_target, int(new_nutri_target * 1.5))
            
            if count > max_keep:
                nutri_subset = nutri_subset.sample(n=max_keep, random_state=42)
                print(f"  Nutri-Score {label}: Reduced from {count} to {max_keep}")
            
            balanced_dfs.append(nutri_subset)
        
        df_balanced = pd.concat(balanced_dfs, ignore_index=True)
        
        # Remove duplicates if any
        df_balanced = df_balanced.drop_duplicates(subset=['product_name'], keep='first')
        
        # Verify new distribution
        print("\n=== Status Check (After Balancing) ===")
        n_products_new = len(df_balanced)
        nutri_counts_new = df_balanced['nutri_score_label'].value_counts()
        green_counts_new = df_balanced['green_score_label'].value_counts()
        
        nutri_target_new = int(n_products_new * min_nutri_pct)
        green_target_new = int(n_products_new * min_green_pct)
        
        print(f"Total products after balancing: {n_products_new}")
        print(f"Nutri-Score target per label: {nutri_target_new}")
        print(f"Green-Score target per label: {green_target_new}\n")
        
        all_pass = True
        for label in ['A', 'B', 'C', 'D', 'E']:
            nutri_count = nutri_counts_new.get(label, 0)
            nutri_pct = (nutri_count / n_products_new * 100) if n_products_new > 0 else 0
            status = "✓" if nutri_count >= nutri_target_new else "❌"
            if nutri_count < nutri_target_new:
                all_pass = False
            print(f"Nutri-Score {label}: {nutri_count} products ({nutri_pct:.1f}%) - {status}")
        
        print()
        for label in ['A', 'B', 'C', 'D', 'E']:
            green_count = green_counts_new.get(label, 0)
            green_pct = (green_count / n_products_new * 100) if n_products_new > 0 else 0
            status = "✓" if green_count >= green_target_new else "❌"
            if green_count < green_target_new:
                all_pass = False
            print(f"Green-Score {label}: {green_count} products ({green_pct:.1f}%) - {status}")
        
        if all_pass:
            print("\n✓ Dataset successfully balanced!")
        else:
            print("\n⚠️  Warning: Could not fully balance dataset with available products.")
            print("   Consider collecting more products from underrepresented categories.")
        
        return df_balanced
    
    else:
        print("\n✓ Dataset already balanced!")
        return df

#### Create Database File

In [16]:
def save_to_excel(products: List[Dict], filename: str = 'food_database.xlsx'):
    """
    Save products to Excel file without formatting
    """
    df = pd.DataFrame(products)
    
    # Reorder columns for clarity
    column_order = [
        'product_name', 'brands', 'categories',
        'energy_100g', 'saturated_fat_100g', 'sugars_100g', 'salt_100g',
        'proteins_100g', 'fiber_100g', 'fvl_percent',
        'nutri_score_value', 'nutri_score_label',
        'green_score_value', 'green_score_label',
        'url'
    ]
    
    df = df[column_order]
    
    # Save to Excel without formatting
    df.to_excel(filename, index=False)
    
    print(f"\n✓ Database saved to {filename}")
    print(f"  Total products: {len(df)}")


In [19]:
def collect_database(categories: List[str], 
                     products_per_category: int = 80,
                     output_file: str = 'food_database.xlsx'):
    all_products = []
    
    for category in categories:
        print(f"\n{'='*60}")
        print(f"Collecting from category: {category}")
        print('='*60)
        
        products = search_products_by_category(
            category, 
            n_products=products_per_category
        )
        all_products.extend(products)
        
        print(f"Total products collected so far: {len(all_products)}")
    
    # Remove duplicates by product name (case-insensitive)
    print(f"\n{'='*60}")
    print("REMOVING DUPLICATES")
    print('='*60)
    df_temp = pd.DataFrame(all_products)
    initial_count = len(df_temp)
    
    # Create a lowercase version for comparison
    df_temp['product_name_lower'] = df_temp['product_name'].str.lower().str.strip()
    df_temp = df_temp.drop_duplicates(subset=['product_name_lower'], keep='first')
    df_temp = df_temp.drop(columns=['product_name_lower'])
    
    final_count = len(df_temp)
    duplicates_removed = initial_count - final_count
    
    print(f"Initial products: {initial_count}")
    print(f"Duplicates removed: {duplicates_removed}")
    print(f"Unique products: {final_count}")
    
    # Balance dataset
    print(f"\n{'='*60}")
    print("BALANCING DATASET")
    print('='*60)
    df_temp = balance_dataset(df_temp)

    # Remove wrong nutri-score computations
    # Keep the default ones only, remove the nutri-scores for products like beverages
    # which do not follow the standard computation
    df_temp = ns.apply_nutriscore_to_database(df_temp)
    df_temp = df_temp[df_temp['calculated_nutri_score'] == df_temp['nutri_score_value']]

    # Save to Excel
    all_products = df_temp.to_dict('records')
    save_to_excel(all_products, output_file)
    
    return all_products

In [20]:
print("START DATA COLLECTION...")
categories_to_collect = ['breads', 'pastas', 'ready-meals', 'soups', 'sauces', 'snacks', 'candies', 'cheese', 'beef', 'chicken', 'fruits']
products = collect_database(
    categories=categories_to_collect,
    products_per_category=50,
    output_file='data/food_database.xlsx'
)

START DATA COLLECTION...

Fetching 50 products from category: breads
Collected 50 products...
Successfully collected 50 products
Total products collected so far: 50

Fetching 50 products from category: pastas
Collected 50 products...
Successfully collected 50 products
Total products collected so far: 100

Fetching 50 products from category: ready-meals
Collected 50 products...
Successfully collected 50 products
Total products collected so far: 150

Fetching 50 products from category: soups
Collected 50 products...
Successfully collected 50 products
Total products collected so far: 200

Fetching 50 products from category: sauces
Collected 50 products...
Successfully collected 50 products
Total products collected so far: 250

Fetching 50 products from category: snacks
Collected 50 products...
Successfully collected 50 products
Total products collected so far: 300

Fetching 50 products from category: candies
Collected 50 products...
Successfully collected 50 products
Total products collec